In [1]:
import os
import cv2
import cv2 as cv
import matplotlib.pyplot as plt
import sys
import imutils
import numpy as np
from scipy.ndimage import interpolation as inter

In [2]:
def imshow(img):
    cv.imshow('Image', img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [21]:
def correct_skew(image, delta=1, limit=5):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2)
        return histogram, score

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        histogram, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, \
              borderMode=cv2.BORDER_REPLICATE)

    return best_angle, rotated

In [25]:
image = cv.imread('gatepass.jpg')
image = imutils.resize(image, height=500)
gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
gray_image = ~gray_image
gray_blur = cv.GaussianBlur(gray_image, (5,5), 0)

In [26]:
edged = cv.Canny(gray_blur, 50,200)
imshow(edged)

In [27]:
angle, rotated = correct_skew(image)
angle

2

In [42]:
edged.shape
row = np.sum(edged, axis=1)
col = np.sum(edged, axis=0)
row.shape, col.shape
row = row // 255
col = col // 255
row_thresh = list(map(lambda x: x if x>150 else 0, row))
col_thresh = list(map(lambda x: x if x>150 else 0, col))
col_thresh = [i for i in range(len(col_thresh)) if col_thresh[i]>150]
row_thresh = [i for i in ]
t = [0]
for i in range(len(col_thresh)-1):
    if abs(col_thresh[i]-col_thresh[i+1]) < 10:
        t[-1] = col_thresh[i]
    else:
        t.append(col_thresh[i])
col_thresh = t
t = [0]
for i in range(len(row_thresh)-1):
    if abs(row_thresh[i]-row_thresh[i+1]) < 10:
        t[-1] = row_thresh[i]
    else:
        t.append(row_thresh[i])
t

[0,
 0,
 176,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 213,
 0,
 0,
 0,
 287,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 268,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 218,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 175,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 165,
 0,
 0,
 0,
 210,
 0,
 0,
 0,
 0,
 0,
 0]

In [9]:
t = edged[:, 200:300]
row = np.sum(t, axis=1) // 255
row_thresh = list(map(lambda x: x if x==100 else 0, row))
row_index = []
for i in range(len(row_thresh)):
    if row_thresh[i]: row_index.append(i)
row_index

[10, 12, 54, 99, 151, 233, 235, 307, 323, 325, 357, 364, 366, 387, 412]

In [10]:
lines = image.copy()
for i in row_index:
    cv.line(lines, (0,i), (350,i), (255,0,0), 2)
imshow(lines)

In [11]:
lines = cv.HoughLines(edged, 1, np.pi/180, 180)
lines[:,0,:].shape

(30, 2)

In [12]:
lined = image.copy()
for rho, theta in lines[:,0,:]:
    if theta < np.pi/18:
        continue
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    cv2.line(lined,(x1,y1),(x2,y2),(0,0,255),2)
imshow(lined)